In [54]:
import pandas as pd
import numpy as np
import pickle

In [55]:
try:
    df_train = pd.read_csv('train_final.csv', index_col=False)
except FileNotFoundError:
    # Handle the situation where the file is not found
    # For example, print an error message and exit the script.
    print("Error: File 'train_final.csv' not found.")
    exit(1)
except pd.errors.EmptyDataError:
    # Handle the situation where the file is empty
    # For example, print an error message and exit the script.
    print("Error: File 'train_final.csv' is empty.")
    exit(1)

In [56]:
X=df_train.drop(columns=['784']).values
y=df_train['784'].values

In [57]:
# Check if X contains numerical data
if not np.issubdtype(X.dtype, np.number):
    # Handle the situation where X does not contain numerical data
    # For example, you could print an error message and exit the script.
    print("Error: X does not contain numerical data.")
    exit(1)

In [58]:
X= X.astype('float32')/255.0

In [59]:
X = X.reshape(-1,28,28,1)

In [60]:
from keras.utils.np_utils import to_categorical
y_cat = to_categorical(y, num_classes=13)

In [61]:
np.random.seed(1212)
import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers
from keras.layers import Input, Dense
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

In [62]:
np.random.seed(7)

In [63]:
model = Sequential()
model.add(Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(13, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [64]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 24, 24, 30)        780       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 12, 12, 30)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 10, 10, 15)        4065      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 5, 5, 15)         0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 5, 5, 15)          0         
                                                                 
 flatten_3 (Flatten)         (None, 375)              

In [65]:
from keras.models import model_from_json

In [66]:
model.fit(X, y_cat, epochs=10, batch_size=32,validation_split=0.2)

Epoch 1/10
3916/3916 [==============================] - 98s 24ms/step - loss: 0.0826 - accuracy: 0.9764 - val_loss: 14.8144 - val_accuracy: 0.1345
Epoch 2/10
3916/3916 [==============================] - 100s 26ms/step - loss: 0.0257 - accuracy: 0.9933 - val_loss: 14.2772 - val_accuracy: 0.1362
Epoch 3/10
3916/3916 [==============================] - 101s 26ms/step - loss: 0.0180 - accuracy: 0.9954 - val_loss: 18.0250 - val_accuracy: 0.1367
Epoch 4/10
3916/3916 [==============================] - 104s 27ms/step - loss: 0.0145 - accuracy: 0.9963 - val_loss: 19.1071 - val_accuracy: 0.1369
Epoch 5/10
3916/3916 [==============================] - 99s 25ms/step - loss: 0.0114 - accuracy: 0.9971 - val_loss: 20.0645 - val_accuracy: 0.1366
Epoch 6/10
3916/3916 [==============================] - 113s 29ms/step - loss: 0.0092 - accuracy: 0.9977 - val_loss: 23.6125 - val_accuracy: 0.1369
Epoch 7/10
3916/3916 [==============================] - 124s 32ms/step - loss: 0.0080 - accuracy: 0.9979 - val_los

In [67]:
model_json = model.to_json()
with open("model_final.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_final.h5")